In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

#np.random.seed(10)

## Measurement probabilities for phase estimation

In [3]:
def prob_zero(omega, phi, t):
    return (np.cos((omega-phi)*t/2)) **2

def prob_one(omega, phi, t):
    return (np.sin((omega-phi)*t/2)) **2

## SMC phase estimation class

In [4]:
class phase_est_smc:

    def __init__(self, omega_star, t0):
        self.omega_star = omega_star
        self.t = t0

    def init_particles(self, num_particles):
        """
        Initializes the particles for SMC.

        Args:
            num_particles: number of particles in the SMC algorithm
        """
        self.num_particles = num_particles
        self.particle_pos = np.linspace(0, 2*np.pi, self.num_particles)
        self.particle_wgts = np.ones(num_particles) * 1/num_particles # uniform weight initialization

    def particles(self, num_measurements=1, threshold=None):
        """
        Runs the SMC algorithm for current experiment until the threshold exceeded

        Args:
            num_measurements: number of measurements per update
            threshold: threshold for n_eff. defaults to self.num_particles/10

        Returns:
            array of particle positions and their corresponding weights
        """

        if threshold is None:
            threshold = self.num_particles/10

        n_eff = None # init None so it will be calculated on first iteration of while loop

        counter = 0 
        while n_eff is None or n_eff >= threshold:
            
            # get measurement
            phi_k = np.random.uniform() * 2 * np.pi
            measure_list = []
            for _ in range(num_measurements):
                r = np.random.uniform()
                if r <= prob_zero(self.omega_star, phi_k, self.t):
                    measure_list.append(0)
                else:
                    measure_list.append(1)

            particle_prob = np.ones(shape=self.num_particles)
            for i in range(num_measurements):
                if measure_list[i] == 0:
                    particle_prob = np.multiply(particle_prob, prob_zero(self.particle_pos, phi_k, self.t))
                else:
                    particle_prob = np.multiply(particle_prob, prob_one(self.particle_pos, phi_k, self.t))

            # bayes update of weights
            self.particle_wgts = np.multiply(self.particle_wgts, particle_prob) # numerator
            norm = np.sum(self.particle_wgts) # denominator
            self.particle_wgts /= norm 
            
            # recalculate n_eff
            n_eff = 1/(np.sum(self.particle_wgts**2))

            counter += 1
            self.update_t()
            
            # if counter % 20 == 0:
            #     print("current iteration {:d}, n_eff = {:f} vs threshold {:f}".format(counter, n_eff, threshold))

        return self.particle_pos, self.particle_wgts

    def update_t(self, factor=9/8):
        """
        Updates time by given factor
        """
        self.t = self.t * factor

    def bootstrap_resample(self):
        """
        Simple bootstrap resampler
        """
        self.particle_pos = np.random.choice(self.particle_pos, size = self.num_particles, p=self.particle_wgts)
        self.particle_wgts = np.ones(self.num_particles) * 1/self.num_particles
        
    def nn_resample(self, bins, edges):
        """
        Convert NN bins to particles.

        Args:
            bins:  [1 x n] np array of bins
            edges: [1 x n+1] np array of bin edges
        """
        
        bins = bins[0]
        edges = edges[0]
        self.particle_pos = (edges[1:] + edges[:-1]) / 2 ## mid point of each bin is the position
        self.particle_wgts = bins
 
        

## Define variables for storing data

In [41]:
num_particles = 100 # number of SMC particles (num of w points)
num_samples = 10000 # number of samples to draw from the particle distribution (to be binned)
num_bins = 100 # number of bins
num_data = 10000 # number of data sets (different w*)

train_data = np.empty((0,num_bins), float)
train_edges = np.empty((0, num_bins+1), float)
train_mean = np.empty((0,), float)
train_std = np.empty((0,), float)

In [42]:
for i in tqdm(range(num_data)): # loop over different experiments
    
    t0 = 0.1
    omega_star = np.random.uniform() * 2 * np.pi
    smc = phase_est_smc(omega_star, t0)
    smc.init_particles(num_particles)
    
    particle_pos, particle_wgts = smc.particles(threshold=num_particles/5, num_measurements=1)
    data = np.random.choice(particle_pos, size = num_samples, p=particle_wgts)
    mean = np.mean(data)
    std = np.std(data)
    
    data = (data-mean)/std
    bins, edges_ = np.histogram(data, num_bins)
    bins = bins/num_samples

    train_mean = np.append(train_mean, mean)
    train_std = np.append(train_std, std)
    train_data = np.append(train_data, [bins], axis=0)
    train_edges = np.append(train_edges, [edges_], axis =0)

#     plt.scatter(particle_pos, particle_wgts)
#     plt.show()
        
#         edge_width = edges_[1] - edges_[0]
#         edges_plot = edges_[:-1]
#         plt.bar(edges_plot, bins, align='edge', width=edge_width)
#         plt.show()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.06it/s]


In [43]:
# np.save("phase_est_data.npy", train_data)
# np.save("phase_est_edges.npy", train_edges)
# np.save("phase_est_mean.npy", train_mean)
# np.save("phase_est_std.npy", train_std)